In [1]:
import os
import mido
import pretty_midi
import random
import numpy as np
import time

In [2]:
raw_dir = os.path.join("..", "inputs", "all-fourbar")
files = os.listdir(raw_dir)
random.shuffle(files)

In [29]:
def fade(current_value, current_time, end_time):
    """
    Scales the current value to zero by the end time.

    Parameters:
    - current_value: The current value to scale.
    - current_time: The current time in ticks
    - end_time: The end time by which the value should reach 0.

    Returns:
    - The scaled value, which linearly decreases to 0 by the end time. If the current time is past the end time, it returns 0.
    """

    scaling_factor = 1 - current_time / (end_time * 8.0)
    scaled_value = current_value * scaling_factor

    return scaled_value

In [30]:
volume = 1.0
runtime = 0.0
with mido.open_output("Disklavier") as output:
    midi = mido.MidiFile(os.path.join(raw_dir, files[0]))
    for msg in midi.play():
        runtime += msg.time
        volume = fade(volume, runtime, midi.length)
        scaled_msg = msg
        if hasattr(msg, "velocity"):
            scaled_msg.velocity = round(scaled_msg.velocity * volume)
            print(f"scaling vel by {volume:.03f} of msg {scaled_msg}")
        output.send(scaled_msg)

0.0 / 14.771504727272726 -> 1.0
0.12167833181818181 / 14.771504727272726 -> 0.9989703289029729
scaling vel by 0.999 of msg note_on channel=0 note=69 velocity=74 time=0.12167833181818181
0.1300699409090909 / 14.771504727272726 -> 0.9988993171031779
scaling vel by 0.998 of msg note_on channel=0 note=67 velocity=70 time=0.00839160909090909
0.16783218181818182 / 14.771504727272726 -> 0.9985797640041004
scaling vel by 0.996 of msg note_on channel=0 note=67 velocity=0 time=0.03776224090909091
0.49510493636363634 / 14.771504727272726 -> 0.9958103038120965
scaling vel by 0.992 of msg note_on channel=0 note=57 velocity=56 time=0.3272727545454545
0.49510493636363634 / 14.771504727272726 -> 0.9958103038120965
scaling vel by 0.988 of msg note_on channel=0 note=60 velocity=54 time=0
0.5160839590909091 / 14.771504727272726 -> 0.995632774312609
scaling vel by 0.984 of msg note_on channel=0 note=64 velocity=65 time=0.020979022727272728
0.5832168318181818 / 14.771504727272726 -> 0.9950646799142492
scal

In [5]:
def scale_vels(midi_data: pretty_midi.PrettyMIDI, scale_factor: float = 1.0):
    """
    Scales the velocities of all notes in a MIDI file by a set amount, with the results capped at 127.

    Args:
        midi_data (pretty_midi.PrettyMIDI): The MIDI file data to be modified.
        scale_factor (float): The factor by which to scale the velocities. A value of 1.0 leaves velocities unchanged,
                            0.5 halves them, etc.
    """
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            new_velocity = int(note.velocity * scale_factor)
            note.velocity = min(new_velocity, 127)

    return midi_data


with mido.open_output("to Max 1") as output:
    for message in mido.MidiFile(os.path.join(raw_dir, files[0])).play():
        output.send(message)

    time.sleep(1)
    scaled_midi = 

In [7]:
def get_velocities(midi_data):
    """
    Analyzes MIDI file velocities.

    Args:
        midi_data (pretty_midi.PrettyMIDI): The MIDI file data.

    Returns:
        (list, list): A 2-element list containing the lowest and highest note velocities,
                    and a list of counts of note velocities broken into 10 bins.
    """
    velocities = []

    for instrument in midi_data.instruments:
        for note in instrument.notes:
            velocities.append(note.velocity)

    if velocities:
        lowest_velocity = min(velocities)
        highest_velocity = max(velocities)
    else:
        lowest_velocity, highest_velocity = 0, 0

    bin_counts, _ = np.histogram(velocities, bins=10, range=(1, 127))

    return [[lowest_velocity, highest_velocity], bin_counts.tolist()]

In [8]:
def scale_midi(midi_data, scale_factor):
    """
    Scales the velocities of all notes in a MIDI file by a set amount, with the results capped at 127.

    Args:
        midi_data (pretty_midi.PrettyMIDI): The MIDI file data to be modified.
        scale_factor (float): The factor by which to scale the velocities. A value of 1.0 leaves velocities unchanged,
                            0.5 halves them, etc.
    """
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            new_velocity = int(note.velocity * scale_factor)
            note.velocity = min(new_velocity, 127)

    return midi_data

In [9]:
factor = 0.25

with mido.open_output("Disklavier") as output:
    for file in files:
        factor = np.random.uniform(0.1, 2.0)
        print(f"scaling by {factor}")
        midi = pretty_midi.PrettyMIDI(os.path.join(raw_dir, file))
        [[v_min, v_max], v_hist] = get_velocities(midi)
        print(f"playing '{file}' ({v_min}, {v_max})")
        print(v_hist)

        for msg in mido.MidiFile(os.path.join(raw_dir, file)).play():
            output.send(msg)

        new_path = os.path.join("..", "outputs", "graveyard", file)
        midi_scaled = scale_midi(midi, factor)
        midi_scaled.write(new_path)

        [[v_min, v_max], v_hist] = get_velocities(midi_scaled)
        print(f"playing '{file}' ({v_min}, {v_max})")
        print(v_hist)

        for msg in mido.MidiFile(new_path).play():
            output.send(msg)

scaling by 0.6571653525723734
playing '20231227-080-02_0084-0096_n00.mid' (40, 71)
[0, 0, 0, 29, 13, 7, 0, 0, 0, 0]
playing '20231227-080-02_0084-0096_n00.mid' (26, 46)
[0, 2, 40, 7, 0, 0, 0, 0, 0, 0]
scaling by 1.991957028007091
playing '20240117-064-05_0060-0075_d03.mid' (29, 80)
[0, 0, 3, 12, 29, 4, 1, 0, 0, 0]
playing '20240117-064-05_0060-0075_d03.mid' (57, 127)
[0, 0, 0, 0, 2, 1, 2, 10, 20, 14]
scaling by 1.339078714126342
playing '20240122-055-01_0174-0192_u04.mid' (21, 83)
[0, 1, 0, 7, 14, 26, 9, 0, 0, 0]
playing '20240122-055-01_0174-0192_u04.mid' (28, 111)
[0, 0, 1, 0, 2, 8, 20, 17, 9, 0]
scaling by 1.2094521369003906
playing '20240213-100-03_0220-0230_u02.mid' (13, 83)
[1, 1, 5, 16, 15, 6, 1, 0, 0, 0]
playing '20240213-100-03_0220-0230_u02.mid' (15, 100)
[0, 2, 1, 8, 13, 14, 4, 3, 0, 0]
scaling by 1.1860603808164492
playing '20240227-076-02_0101-0113_d04.mid' (61, 93)
[0, 0, 0, 0, 1, 17, 81, 2, 0, 0]
playing '20240227-076-02_0101-0113_d04.mid' (72, 110)
[0, 0, 0, 0, 0, 2, 14

KeyboardInterrupt: 